# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [37]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath=os.path.join('..', 'output', 'Weather_data')
weather=pd.read_csv(csvpath)
weather=weather.drop(columns="Unnamed: 0")
weather.head()

,City,lat,lon,Max Temp,Cloudiness,Humidity,Wind speed,Country
0,Menzel Salem,35.86,8.48,77.00,40,33,11.41,TN
1,Kapaa,22.08,-159.32,77.00,75,83,4.70,US
2,Airai,-8.93,125.41,61.25,58,96,3.33,TL
3,Ushuaia,-54.80,-68.30,51.80,40,73,5.82,AR
4,Souillac,-20.52,57.52,75.20,40,88,8.05,MU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(g_key)

In [4]:

locations = weather[["lat", "lon"]].astype(float)
humidity=weather['Humidity'].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
hotel_df=weather.loc[(weather["Max Temp"]<90) & (weather["Max Temp"]>65) & (weather['Humidity']<=10)]
hotel_df=hotel_df.reset_index(drop=True)
hotel_df

,City,lat,lon,Max Temp,Cloudiness,Humidity,Wind speed,Country
0,Mahrīz,31.59,54.43,69.8,0,9,6.93,IR
1,Mount Isa,-20.73,139.50,77.0,3,10,9.17,AU
2,Faya,18.39,42.45,80.6,20,9,6.93,SA


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
hotel_df["Hotel Name"]=""
url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for i, row in hotel_df.iterrows():
    
    lat=row["lat"]
    lon=row['lon']

    params = {
        "location": f"{lat}, {lon}",
        "radius": 5000,
        "types": "lodging",
        "key": g_key
    }
    response=requests.get(url, params=params)
    name=response.json()['results'][0]['name']
    hotel_df.loc[i, "Hotel Name"]=name
hotel_df

,City,lat,lon,Max Temp,Cloudiness,Humidity,Wind speed,Country,Hotel Name
0,Mahrīz,31.59,54.43,69.8,0,9,6.93,IR,Mehrgard dwelling/سرای مهرگرد
1,Mount Isa,-20.73,139.50,77.0,3,10,9.17,AU,ibis Styles Mt Isa Verona
2,Faya,18.39,42.45,80.6,20,9,6.93,SA,مقبرة ال مجمل


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [48]:
# Add marker layer ontop of heat map


hotel_layer = gmaps.marker_layer(
    locations, 
    info_box_content=hotel_info
)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))